In [1]:
#import librosa
import os
from matplotlib import pyplot as plt
import IPython.display as ipd
import pandas as pd
from pydub import AudioSegment
import ffprobe
import ffmpeg
import numpy as np
import wave
import os
from pydub import AudioSegment
import numpy as np
from teager_py import Teager
import pydub

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


Bad key text.latex.preview in file /software/python-anaconda-2020.11-el8-x86_64/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /software/python-anaconda-2020.11-el8-x86_64/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /software/python-anaconda-2020.11-el8-x86_64/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig

In [2]:
import zipfile

# zip file handler  
zip = zipfile.ZipFile('/home/xuranzeng/thesis/ReleasedDataset.zip')

# list available files in the container
filename = zip.namelist()

# clean path and file names
audioname = (item for item in filename if item.endswith('mp3')) 
data = pd.DataFrame(list(audioname))
data = pd.concat([data, data[0].str.split('/', 4, expand=True)], axis=1) # split path 
data = pd.concat([data, data[1].str.split('_', 2, expand=True)], axis=1) # split company and day
data.columns=["Path","Basic", "Company_Day", "Type", "Filename","Company","Day"] # rename column
data = data[['Path','Company',"Day","Filename"]] # delete useless columns

# path
base_path='/home/xuranzeng/thesis/EC/'
data['Complete_Path'] = base_path + data['Path'].astype(str)
data = data[['Complete_Path','Company',"Day","Filename"]] # delete useless columns

In [69]:
data.head(10)

,Complete_Path,Company,Day,Filename
0,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_15_1.mp3
1,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_16_1.mp3
2,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_1_14.mp3
3,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_2_11.mp3
4,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_10_5.mp3
5,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_9_11.mp3
6,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_15_4.mp3
7,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_1_45.mp3
8,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_12_1.mp3
9,/home/xuranzeng/thesis/EC/ReleasedDataset_mp3/...,Booking Holdings Inc,20170509,Daniel J. Finnegan_6_13.mp3


In [46]:
company_path=data[['Company','Day']].drop_duplicates()

In [48]:
#company_path.to_csv('/home/xuranzeng/thesis/symbol_company_day.csv')

In [63]:
company_symbol= pd.read_csv('/home/xuranzeng/thesis/symbol_company_day.csv')

In [64]:
company_symbol

,Unnamed: 0,Company,Day,Symbol
0,0,3M Company,20170725,MMM
1,1,3M Company,20170425,MMM
2,2,A.O. Smith Corp,20170726,AOS
3,3,Abbott Laboratories,20171018,ABT
4,4,AbbVie Inc.,20171027,ABBV
...,...,...,...,...
566,566,Xerox,20170801,XRX
567,567,Xilinx,20170426,XLNX
568,568,XL Group,20171024,XL
569,569,Yum! Brands Inc,20170803,YUM


In [42]:
#data.to_csv('/home/xuranzeng/thesis/EC_path.csv')

In [ ]:
# feature extraction

In [9]:
path = pd.read_csv('/home/xuranzeng/thesis/EC_path.csv')
path['Day']=pd.to_numeric(path.Day)
company_name = "3M Company"
day_name = 20170425
sample = path.loc[(path["Company"]==company_name) & (path["Day"]==day_name)].reset_index(drop=True)

In [23]:
company_name = "3M Company"
day_name = 20170425
sample = path.loc[(path["Company"]==company_name) & (path["Day"]==day_name)].reset_index(drop=True)

In [26]:
sample = sample[["Complete_Path"]]
sample = sample.rename(columns={'Complete_Path': 'mp3path'})

In [28]:
#sample.to_csv('/home/xuranzeng/TSFSER/code/data/sample_EC.csv')

In [29]:
# GeMAPS
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,   
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

In [30]:
def get_GeMAPS(path):
    GeMAPS = []
    for i in range(0,len(path)):
        filepath = path.loc[i,'mp3path']
        audio = AudioSegment.from_mp3(filepath)
        sr = audio.frame_rate
        audio = audio.get_array_of_samples()
        df = smile.process_signal(audio,sr)
        GeMAPS.append(pd.DataFrame(df.mean(axis=0)).T)
        print('processing:',i)
    

    GeMAPS = pd.concat(GeMAPS).reset_index()
    return GeMAPS

In [31]:
GeMAPS_sample = get_GeMAPS(sample)

processing: 0
processing: 1
processing: 2
processing: 3
processing: 4
processing: 5
processing: 6
processing: 7
processing: 8
processing: 9
processing: 10
processing: 11
processing: 12
processing: 13
processing: 14
processing: 15
processing: 16
processing: 17
processing: 18
processing: 19
processing: 20
processing: 21
processing: 22
processing: 23
processing: 24
processing: 25
processing: 26
processing: 27
processing: 28
processing: 29
processing: 30
processing: 31
processing: 32
processing: 33
processing: 34
processing: 35
processing: 36
processing: 37
processing: 38
processing: 39
processing: 40
processing: 41
processing: 42
processing: 43
processing: 44
processing: 45
processing: 46
processing: 47
processing: 48
processing: 49
processing: 50
processing: 51
processing: 52
processing: 53
processing: 54
processing: 55
processing: 56
processing: 57
processing: 58
processing: 59
processing: 60
processing: 61
processing: 62
processing: 63
processing: 64
processing: 65
processing: 66
proce

In [33]:
result = np.mean(GeMAPS_sample,axis=0)

In [34]:
result

index                             0.000000
Loudness_sma3                     4.690456
alphaRatio_sma3                  -3.991197
hammarbergIndex_sma3             15.528037
slope0-500_sma3                  -0.073613
slope500-1500_sma3               -0.001370
spectralFlux_sma3                 3.028974
mfcc1_sma3                       -6.776380
mfcc2_sma3                        3.093613
mfcc3_sma3                        2.390142
mfcc4_sma3                        5.421530
F0semitoneFrom27.5Hz_sma3nz       0.000000
jitterLocal_sma3nz                0.000000
shimmerLocaldB_sma3nz             0.000000
HNRdBACF_sma3nz                   0.000000
logRelF0-H1-H2_sma3nz             0.000000
logRelF0-H1-A3_sma3nz             0.000000
F1frequency_sma3nz              614.705933
F1bandwidth_sma3nz              640.605286
F1amplitudeLogRelF0_sma3nz     -201.000000
F2frequency_sma3nz             1503.727295
F2bandwidth_sma3nz              766.380249
F2amplitudeLogRelF0_sma3nz     -201.000000
F3frequency

In [137]:
#GeMAPS_0105.to_csv('/home/xuranzeng/thesis/GeMAPS_0105.csv')

In [145]:
features = pd.concat([sample, GeMAPS_0105], axis=1)

In [147]:
#features.to_csv('/home/xuranzeng/thesis/features_0105.csv')

In [35]:
import json
import csv

In [36]:
with open('/home/xuranzeng/thesis_git/code/data/EC_feature.json') as json_file:
    data = json.load(json_file)

In [41]:
pd.DataFrame(data)['gemaps']

0        [6.287932395935059, -6.639183044433594, 18.396...
1        [6.253244400024414, -6.663923263549805, 18.445...
2        [6.120508193969727, -6.574212074279785, 18.131...
3        [6.2892913818359375, -6.71579122543335, 18.561...
4        [6.296769142150879, -6.597139358520508, 18.456...
                               ...                        
89838    [6.285790920257568, -6.669467926025391, 18.443...
89839    [6.296391010284424, -6.643906116485596, 18.443...
89840    [6.293277263641357, -6.729918003082275, 18.471...
89841    [6.290667533874512, -6.693284034729004, 18.534...
89842    [6.301898002624512, -6.689201354980469, 18.515...
Name: gemaps, Length: 89843, dtype: object

In [ ]:
# text path

In [5]:
import zipfile

# zip file handler  
zip = zipfile.ZipFile('/home/xuranzeng/thesis/ReleasedDataset.zip')

# list available files in the container
filename = zip.namelist()

# clean path and file names
audioname = (item for item in filename if item.endswith('txt')) 
data = pd.DataFrame(list(audioname))
data = pd.concat([data, data[0].str.split('/', 4, expand=True)], axis=1) # split path 
data = pd.concat([data, data[1].str.split('_', 2, expand=True)], axis=1) # split company and day
data.columns=["Path","Basic", "Company_Day", "Type", "Filename","Company","Day"] # rename column
data = data[['Path','Company',"Day","Filename"]] # delete useless columns

# path
base_path='/home/xuranzeng/thesis/EC/'
data['Complete_Path'] = base_path + data['Path'].astype(str)
data = data[['Complete_Path','Company',"Day","Filename"]] # delete useless columns

In [27]:
import zipfile

# zip file handler  
zip = zipfile.ZipFile('/home/xuranzeng/thesis/ReleasedDataset.zip')

# list available files in the container
filename = zip.namelist()

audioname = (item for item in filename if item.endswith('txt'))
data = pd.DataFrame(list(audioname))

data = pd.concat([data, data[0].str.split('/', 2, expand=True)], axis=1) # split path 
data = pd.concat([data, data[1].str.split('_', 2, expand=True)], axis=1) # split company and day

data.columns=["Path","Basic", "Company_Day", "Filename","Company","Day"] # rename column
data = data[['Path','Company',"Day","Filename"]] # delete useless columns

# path
base_path='/home/xuranzeng/thesis/EC/'
data['Complete_Path'] = base_path + data['Path'].astype(str)
data = data[['Complete_Path','Company',"Day","Filename"]] # delete useless columns

In [29]:
#data.to_csv('/home/xuranzeng/thesis/text_path.csv')

In [71]:
# get paragraph and sentence
audio = pd.read_csv('/home/xuranzeng/thesis/EC_path.csv').iloc[: , 1:]
audio['Day']=pd.to_numeric(audio.Day)
audio = pd.concat([audio, audio['Filename'].str.split('_', 3, expand=True)], axis=1) # split path 
audio.columns=["Complete_Path","Company", "Day", "Filename","Speaker","Paragraph","Sentence"] # rename column

audio.loc[:,'Sentence']=audio.loc[:,'Sentence'].str.replace(".mp3", "")
audio['Paragraph']=pd.to_numeric(audio.Paragraph)
audio['Sentence']=pd.to_numeric(audio.Sentence)


audio = audio.sort_values(["Company","Day","Paragraph", "Sentence"], ascending = (True,True,True,  True))

In [74]:
#audio.to_csv('/home/xuranzeng/thesis/EC_path2.csv')

In [119]:
# merge text audio

data = pd.read_csv('/home/xuranzeng/thesis/text_path.csv').iloc[: , 1:]
data = data[['Complete_Path','Company',"Day"]] # delete useless columns
data.columns=["Text_Path","Company", "Day"] # rename column

audio = pd.read_csv('/home/xuranzeng/thesis/EC_path2.csv').iloc[: , 1:-2]


audio['Day']=pd.to_numeric(audio.Day)
data['Day']=pd.to_numeric(data.Day)

merged_path = pd.merge(audio, data, on = ['Day','Company']).dropna()

In [121]:
#merged_path.to_csv('/home/xuranzeng/thesis/EC_path2.csv')

In [66]:
path = pd.read_csv('/home/xuranzeng/thesis/EC_path2.csv').iloc[: , 1:]
path['Day']=pd.to_numeric(path.Day)
company_name = "Booking Holdings Inc"
day_name = 20170509
sample = path.loc[(path["Company"]==company_name) & (path["Day"]==day_name)].reset_index(drop=True)

In [67]:
text_path = path.drop_duplicates(subset=['Text_Path']).reset_index()

In [95]:
i=6
company_name = text_path.loc[i,'Company']
day_name = text_path.loc[i,'Day']
sample = path.loc[(path["Company"]==company_name) & (path["Day"]==day_name)].reset_index(drop=True)
text = sample.loc[0,'Text_Path']
with open(text) as f:
    contents = f.read()
    #print(contents)
    sen = contents.split('\n')[:-1]
    
    
    if len(sen)==len(sample):
        sample['Content']=sen
        #print(sen)
        
    else:
        print(len(sen))
        print(len(sample))
        #print(i)
    #print(len(sen))
    

353
359


In [91]:
print(day_name)
print(company_name)

20171102
AES Corp


In [85]:
sen

['Thanks, Ajita',
 'Sales for the second quarter of $738 million were 11% higher than the previous year',
 'Net earnings in the second quarter of $92 million increased 6% from 2016. Second quarter earnings per share of $0.53 increased 8% compared with 2016. Sales in our North America segment of $471 million increased 9% compared with the second quarter of 2016. The increase in sales was primarily due to higher volumes of commercial water heaters and boilers in the U.S',
 'and pricing actions in August 2016 related to steel cost increases and inflationary pressure on other costs',
 'Aquasana, acquired in August of 2016, added $13 million to our North America segment sales',
 'Rest of World segment sales of $273 million increased 14% compared with 2016. China sales increased 20% in local currency, driven by higher demand for our consumer products in the region, led by water treatment and air purification products, and pricing actions due to higher steel and other costs, which were announ

['Thank you, Ingrid',
 'Good morning, everyone',
 'In Q4, Abiomed had another strong quarter of continued adoption of the Impella family of devices with $124.7 million in revenue, representing growth of 33%',
 'patient utilization increased by 30%, driven by continued growth in both Protected PCI and emergent patients, which grew 26% and 37%, respectively',
 'For the full fiscal year 2017, Abiomed generated $445.3 million in revenue, representing a net increase of $116 million or growth of 35%',
 'Abiomed remains one of the fastest-growing GAAP profitable medical device companies and fiscal 2017 was another successful year, as we achieved our corporate goals',
 'We received FDA PMA approval for cardiogenic shock, Impella CP approval for high-risk PCI, IDE approval for our STEMI Door to Unloading study, and approval in Japan',
 'We achieved CMS approval for dedicated DRGs and expanded our clinical evidence to over 400 publications',
 'We ramped up our distribution in the U.S',
 'and Ger

In [6]:
i=2

In [14]:
company_name = text_path.loc[i,'Company']
day_name = text_path.loc[i,'Day']
sample = path.loc[(path["Company"]==company_name) & (path["Day"]==day_name)].reset_index(drop=True)
text = sample.loc[0,'Text_Path']
with open(text) as f:
    contents = f.read()
    #print(contents)
    sen = contents.split('.')[:-1]

In [18]:
import re

In [60]:
type(line)

list

In [61]:
line=[i.split('.') for i in contents.splitlines()]

In [62]:
flat_list = []
for sublist in line:
    for item in sublist:
        flat_list.append(item)